In [ ]:
import pandas as pd
from datasets import DatasetDict, Dataset
import re

: 

In [ ]:


# Load and clean data
data = pd.read_excel(r"D:\jincheng_project\data.xlsx")  

# Remove special chars
data["title"] = data["title"].apply(lambda x: re.sub(r'[\u200b-\u200f\u202a-\u202e]', '', str(x)))
data["content"] = data["content"].apply(lambda x: re.sub(r'[\u200b-\u200f\u202a-\u202e]', '', str(x)))

# --- Define a PROMPT TEMPLATE ---
def format_question(row):
    return (
        f"根据以下信息回答问题：\n"
        f"标题: {row['title']}\n"
        f"内容: {row['content']}\n"
        f"问题类型: {row['type_name']}\n"
        f"地区: {row['af_address']}\n"
        f"请问如何处理此问题？"
    )

# Apply formatting
data["question"] = data.apply(format_question, axis=1)  # Question
data["response"] = data["jian_cha_info"]               # Answer

# Drop missing outputs
data = data.dropna(subset=["response"])  # or fill with "无回复"

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(data[["question", "response"]])
# dataset = dataset.train_test_split(test_size=0, seed=42)



d:\jincheng_project\jincheng\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
formatted_data = pd.DataFrame({
    'Question': dataset['question'],
    'Response': dataset['response']
})

# Convert to HuggingFace Dataset
formatted_dataset = Dataset.from_pandas(formatted_data)

# Create DatasetDict with only train split
final_dataset = DatasetDict({
    'train': formatted_dataset
})

# Verify the structure
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Response'],
        num_rows: 39578
    })
})
